In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [ ]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

In [ ]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

# Create ToolNode

In [ ]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks, get_audio_features
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, find_similar_artists, find_top_tracks, search_tool, get_audio_features]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# System Prompt

In [ ]:
from langchain_core.messages import SystemMessage
from prompts import Prompts

system_message = SystemMessage(Prompts.SYSTEM)


# First Message

In [ ]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
llm_response = llm_with_tools.invoke([system_message, human_message])

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [ ]:
llm_response.pretty_print()

In [ ]:
plan = llm_response.tool_calls[0]["args"]
plan

# chat_prompt_template holds all messages (Human, AI, Tool)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template: ChatPromptTemplate = system_message + human_message + llm_response
messages = chat_prompt_template.format_messages()
messages

# Check for Tools in Message

In [ ]:
from typing import Any, Callable, List
from langchain_core.messages import BaseMessage

def check_tool_in_messages(messages: List[BaseMessage], tools: List[Callable[..., Any]]) -> List[str]:
    tools_in_ai_message = []
    tools_by_name = {tool.name: tool for tool in tools}
    for tool_call in messages[-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        tools_in_ai_message.append(tool)
        # observation = tool.invoke(tool_call["args"])
        # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return tools_in_ai_message

# Tool Call (should be validate_plan())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
tool_message = tool_response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

# Send Messages with Tool result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(chat_prompt_template.format_messages())
llm_response.pretty_print()

# Validate correct tool call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
tool_response = tool_node.invoke({"messages": messages})

In [ ]:
for tool_message in tool_response["messages"]:
    tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result Results to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

# Validate correct tool call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'get_artists_from_playlist'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in tool_response["messages"]:
    tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

# Check Tool Call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'find_similar_artists'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call (find_similar_artists())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call (filter_artists())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "filter_artists":
    raise ValueError("should be tool message calling 'filter_artists'")

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_top_tracks":
    raise ValueError("should be tool message calling 'find_top_tracks'")

# Tool Call

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()